d-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Oversampling and Undersampling Classes

**Objective**: *Demonstrate how to bootstrap records based on their label values.*

In this video we will demonstrate how to bootstrap training set records into a new training set based on the target class distribution to ensure a more balanced distribution in the training set.

In [0]:
%run "../../Includes/Classroom-Setup"

Mounting course-specific datasets to /mnt/training ... Datasets are already mounted to /mnt/training from s3a://databricks-corp-training/common

Out[2]: DataFrame[]

res1: Boolean = false

res2: Boolean = false

res3: Boolean = false

We also need to load the below library.

In [0]:
%pip install imbalanced-learn

Python interpreter will be restarted.
Collecting imbalanced-learn
 Downloading imbalanced_learn-0.7.0-py3-none-any.whl (167 kB)
Requirement already satisfied: scipy>=0.19.1 in /databricks/python3/lib/python3.7/site-packages (from imbalanced-learn) (1.4.1)
Requirement already satisfied: numpy>=1.13.3 in /databricks/python3/lib/python3.7/site-packages (from imbalanced-learn) (1.18.1)
Requirement already satisfied: joblib>=0.11 in /databricks/python3/lib/python3.7/site-packages (from imbalanced-learn) (0.14.1)
Collecting scikit-learn>=0.23
 Downloading scikit_learn-0.24.0-cp37-cp37m-manylinux2010_x86_64.whl (22.3 MB)
Collecting threadpoolctl>=2.0.0
 Downloading threadpoolctl-2.1.0-py3-none-any.whl (12 kB)
Installing collected packages: threadpoolctl, scikit-learn, imbalanced-learn
 Attempting uninstall: scikit-learn
 Found existing installation: scikit-learn 0.22.1
 Not uninstalling scikit-learn at /databricks/python3/lib/python3.7/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-4d72d00d-9c4e-441a-90a0-9c9af5d4c55e
 Can't uninstall 'scikit-learn'. No files were found to uninstall.
Successfully installed imbalanced-learn-0.7.0 scikit-learn-0.24.0 threadpoolctl-2.1.0
Python interpreter will be restarted.

## Prepare data

We will once again be using the `adsda.ht_user_metrics_lifestyle` table that we previously created, and predicting which lifestyle class users fall into.

In [0]:
ht_lifestyle_pd_df = spark.table("adsda.ht_user_metrics_lifestyle").toPandas()

In [0]:
ht_lifestyle_pd_df['lifestyle'].unique().tolist()

--------------------------------------------------------------------------- 
 KeyError Traceback (most recent call last)
 /databricks/python/lib/python3.7/site-packages/pandas/core/indexes/base.py in get_loc (self, key, method, tolerance) 
 2645 try : 
 -> 2646 return self . _engine . get_loc ( key ) 
 2647 except KeyError : 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 KeyError : 'lifestyle'

During handling of the above exception, another exception occurred:

 KeyError Traceback (most recent call last)
 <command-1273035449082521> in <module> 
 ----> 1 ht_lifestyle_pd_df [ 'lifestyle' ] . unique ( ) . tolist ( ) 

 /databricks/python/lib/python3.7/site-packages/pandas/core/frame.py in __getitem__ (self, key) 
 2798 if self . columns . nlevels > 1 : 
 2799 return self . _getitem_multilevel ( key ) 
 -> 2800 indexer = self . columns . get_loc ( key ) 
 2801 if is_integer ( indexer ) : 
 2802 indexer = [ indexer ] 

 /databricks/python/lib/python3.7/site-packages/pandas/core/indexes/base.py in get_loc (self, key, method, tolerance) 
 2646 return self . _engine . get_loc ( key ) 
 2647 except KeyError : 
 -> 2648 return self . _engine . get_loc ( self . _maybe_cast_indexer ( key ) ) 
 2649 indexer = self . get_indexer ( [ key ] , method = method , tolerance = tolerance ) 
 2650 if indexer . ndim > 1 or indexer . size > 1 : 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/index.pyx in pandas._libs.index.IndexEngine.get_loc () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 pandas/_libs/hashtable_class_helper.pxi in pandas._libs.hashtable.PyObjectHashTable.get_item () 

 KeyError : 'lifestyle'

In this example, we want to know whether each user is "sedentary" or "active" (any of the other three lifestyle classes). 

Therefore we will convert the four categories into two numeric classes, 0 and 1. We will accomplish this using a list comprehension and Pandas.

In [0]:
ht_lifestyle_pd_df['lifestyle_num'] = [0 if x=='Sedentary' else 1 for x in ht_lifestyle_pd_df['lifestyle']]

We can check how many of each category there are:

In [0]:
ht_lifestyle_pd_df['lifestyle_num'].value_counts()

In [0]:
X = ht_lifestyle_pd_df.drop("lifestyle", axis=1).drop("lifestyle_num", axis=1)
                           
y = ht_lifestyle_pd_df['lifestyle_num']

## Bootstrap sample the minority class


We now have a dataset with a slightly imbalanced target class: class 1 is almost 10 times bigger than class 0. (This is not a very major imbalance, relatively speaking.) We will attempt to balance the dataset using the bootstrap method.

We will perform random oversampling, where we randomly duplicate examples in the minority class by sampling with replacement.

We set the sampling strategy to `minority` which will make the minority class the same size as the majority class.

In [0]:
from imblearn.over_sampling import RandomOverSampler

oversample = RandomOverSampler(sampling_strategy='minority')

Fit and apply the oversample transformation:

In [0]:
X_over, y_over = oversample.fit_resample(X, y)

In [0]:
y_over.value_counts()

## Undersample the majority class

Next, we will perform random undersampling, where we randomly delete examples from the majority class.

Setting the sampling strategy to `majority` will make the majority class the same size as the minority class.

In [0]:
from imblearn.under_sampling import RandomUnderSampler

undersample = RandomUnderSampler(sampling_strategy='majority')

In [0]:
X_under, y_under = undersample.fit_resample(X, y)

In [0]:
y_under.value_counts()

Instead of simply saying that we want the majority class to be the same size as the minority, we can specify a ratio:

In [0]:
undersample_2 = RandomUnderSampler(sampling_strategy=0.75)

In [0]:
X_under2, y_under2 = undersample_2.fit_resample(X, y)

In [0]:
y_under2.value_counts()

-sandbox
&copy; 2020 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="http://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="http://help.databricks.com/">Support</a>